In [ ]:
from datetime import datetime, timedelta
import sys
import re

In [ ]:
import pandas as pd
from ics import Calendar, Event
from pytz import timezone

In [ ]:
pattern = re.compile(r'(\d+)[^\d]+')

In [ ]:
def parse_date(row, year=2020):
    try:
        day = int(pattern.findall(row.Date)[0])
                
        month = int(row.Month)
        return datetime(year=year, month=month, day=day)
    except Exception as e:
#         print(year, month, day, file=sys.stderr)
        print(row, file=sys.stderr)
        raise e

In [ ]:
def s_to_str2(s):
    return '\n\n'.join(f'{key}: {value}' for key, value in s.items() if not pd.isnull(value))

In [ ]:
basename = '2020-reading-schedule-{lang}.csv'

In [ ]:
lang = 'english'

In [ ]:
df = pd.read_csv(basename.format(lang=lang))

In [ ]:
df['date'] = df.apply(parse_date, axis=1)

In [ ]:
# all of them should differs by a day
s = df.date.diff().value_counts()
assert s.shape[0] == 1
s

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
pacific = timezone('US/Pacific')

In [ ]:
c = Calendar()
for _, row in df.iterrows():
    for key, value in row[['Reading A', 'Reading B', 'Study']].items():
        if not pd.isnull(value):
            c.events.add(Event(
                name=f'{key}: {value}',
                begin=pacific.localize(row.date),
                duration=timedelta(days=1),
            ))

In [ ]:
with open(basename[:-3].format(lang=lang) + 'ics', 'w') as f:
    f.writelines(c)